In [56]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc 
from functools import reduce
import wfdb#https://github.com/MIT-LCP/wfdb-python
from wfdb import processing
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(0)
print (torch.cuda.current_device())

0


In [16]:
#Beats generation,
#we defined a single ECG beat image by centering the Q-wave peak signal while
#excluding the first and the last 20 ECG signals from the previous and afterward Q-wave peak signal
#https://github.com/MIT-LCP/wfdb-python/blob/master/demo.ipynb
#https://archive.physionet.org/physiobank/database/html/mitdbdir/mitdbdir.htm
#http://www.tara.tcd.ie/bitstream/handle/2262/17623/automatic.pdf?sequence=1
def labeltotext(val):
    if val in ['N','L','R','e','j'] :
        return 0 #N
    elif val in ['A','a','J','S']:
        return 1 #S
    elif val in ['V','E']:
        return 2 #V
    elif val == 'F':
        return 3 #F
    elif val in ['/','f','Q']:
        return 4 #Q
    else: 
        pass
    
rootdir = '/data/fjsdata/physionet/MIT-BIH/mitdb/'
right_len = 180 #right sample length around of peak value of QRS
left_len = 20 #left sample length around of peak value of QRS
#get trainset
trData = [] #[QRS value, label]
for bt in [101,106,108,109,112,114,115,116,118,119,122,124,201,203,205,207,208,209,215,220,223,230]:#22 records for train
    file = os.path.join(rootdir,str(bt))
    try:
        annotation = wfdb.rdann(file, 'atr') 
        qrs_spl = annotation.sample #numpy.ndarray
        qrs_sym = annotation.symbol #list
        record = wfdb.rdrecord(file)
        signal = record.p_signal #numpy.ndarray
        max_len = record.sig_len #length of samples
        lead_name =  record.sig_name #names of lead channels,list
        for i in range(annotation.ann_len):
            if qrs_sym[i] in ['N','L','R','e','j','A','a','J','S','V','E','F','/','f','Q']:#seven diseases samples
                pos = qrs_spl[i] #corresponding position of peak value of QRS
                if pos+right_len<=max_len and pos-left_len>=0:
                    max_idx = pos+right_len#np.min([max_len, pos+trunc_len])
                    min_idx = pos-left_len#np.max([0, pos-trunc_len])
                    #for j, val in enumerate(lead_name):
                        #QRS_value = signal[:,j][min_idx:max_idx]
                        #data.append([QRS_value,labeltotext(qrs_sym[i]),val])#[QRS value, label, lead name]
                    QRS_value = signal[:,0][min_idx:max_idx] #only one lead
                    trData.append([QRS_value,labeltotext(qrs_sym[i])])
    except:
        pass
    
trData = pd.DataFrame(np.array(trData))
X_train = pd.DataFrame(trData[0].values.tolist())
y_train = trData[1]
print('The shape of trainset is: (%d,%d)'%(X_train.shape[0],X_train.shape[1]))
print(trData[1].value_counts())
#get testset
teData = [] #[QRS value, label]
for bt in [100,103,105,111,113,117,121,123,200,202,210,212,213,214,219,221,222,228,231,232,233,234]:#22 records for test
    file = os.path.join(rootdir,str(bt))
    try:
        annotation = wfdb.rdann(file, 'atr') 
        qrs_spl = annotation.sample #numpy.ndarray
        qrs_sym = annotation.symbol #list
        record = wfdb.rdrecord(file)
        signal = record.p_signal #numpy.ndarray
        max_len = record.sig_len #length of samples
        lead_name =  record.sig_name #names of lead channels,list
        for i in range(annotation.ann_len):
            if qrs_sym[i] in ['N','L','R','e','j','A','a','J','S','V','E','F','/','f','Q']:#seven diseases samples
                pos = qrs_spl[i] #corresponding position of peak value of QRS
                if pos+right_len<=max_len and pos-left_len>=0:
                    max_idx = pos+right_len#np.min([max_len, pos+trunc_len])
                    min_idx = pos-left_len#np.max([0, pos-trunc_len])
                    #for j, val in enumerate(lead_name):
                        #QRS_value = signal[:,j][min_idx:max_idx]
                        #data.append([QRS_value,labeltotext(qrs_sym[i]),val])#[QRS value, label, lead name]
                    QRS_value = signal[:,0][min_idx:max_idx] #only one lead
                    teData.append([QRS_value,labeltotext(qrs_sym[i])])
    except:
        pass

teData = pd.DataFrame(np.array(teData))
X_test = pd.DataFrame(teData[0].values.tolist())
y_test = teData[1]
print('The shape of testset is: (%d,%d)'%(X_test.shape[0],X_test.shape[1]))
print(teData[1].value_counts())

The shape of trainset is: (51009,200)
0    45854
2     3788
1      944
3      415
4        8
Name: 1, dtype: int64
The shape of testset is: (49698,200)
0    44245
2     3221
1     1837
3      388
4        7
Name: 1, dtype: int64


In [79]:
#model: faiss+index
# buliding index of trainset
tstart = time.time()
cpu_index = faiss.IndexFlatL2(right_len+left_len) #
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
gpu_index.add(np.ascontiguousarray(X_train, dtype=np.float32)) #add data(must be float32) to index
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))
#performance
X_test = np.ascontiguousarray(X_test, dtype=np.float32)
scores, neighbors = gpu_index.search(X_test, k=1) #return top1
y_pred = []
for i in neighbors.flatten():
    y_pred.append(np.array(y_train)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(y_test.tolist(), y_pred))
#confusion matrix
labels = list(set(y_pred))
cm = confusion_matrix(y_test.tolist(), y_pred, labels=labels ) #labels=['N','S','V','F','Q']
print (cm)
print ('Specificity: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of S: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of V: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of F: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of Q: %.6f'%float(cm[4][4]/np.sum(cm[4])))

Completed buliding index in 1 seconds
Accuracy: 0.826814
[[38424  2401  2534   854    32]
 [ 1758    42     8    29     0]
 [  495    79  2624    23     0]
 [  260     0   127     1     0]
 [    2     0     5     0     0]]
Specificity: 0.868437
Sensitivity of S: 0.022863
Sensitivity of V: 0.814654
Sensitivity of F: 0.002577
Sensitivity of Q: 0.000000


In [80]:
#model: LSH
from sklearn.neighbors import LSHForest
lshf = LSHForest(min_hash_match=4, n_neighbors=20, n_candidates=10, n_estimators =10, random_state=42) #hashcode=32
lshf.fit(X_train)  
distances, indices = lshf.kneighbors(X_test, n_neighbors=1)#top1
y_pred = []
for i in indices.flatten():
    y_pred.append(np.array(y_train)[i]) #label of top1
print ( 'Accuracy: %.6f'%accuracy_score(y_test.tolist(), y_pred))
#confusion matrix
labels = list(set(y_pred))
cm = confusion_matrix(y_test.tolist(), y_pred, labels=labels ) #labels=['N','S','V','F','Q']
print (cm)
print ('Specificity: %.6f'%float(cm[0][0]/np.sum(cm[0])))
print ('Sensitivity of S: %.6f'%float(cm[1][1]/np.sum(cm[1])))
print ('Sensitivity of V: %.6f'%float(cm[2][2]/np.sum(cm[2])))
print ('Sensitivity of F: %.6f'%float(cm[3][3]/np.sum(cm[3])))
print ('Sensitivity of Q: %.6f'%float(cm[4][4]/np.sum(cm[4])))

/root/miniconda3/lib/python3.7/site-packages/sklearn/neighbors/approximate.py:258: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


Accuracy: 0.819872
[[38020  1574  3375  1265    11]
 [ 1425    56   311    45     0]
 [  487    50  2664    20     0]
 [  345     1    37     5     0]
 [    4     0     2     0     1]]
Specificity: 0.859306
Sensitivity of S: 0.030484
Sensitivity of V: 0.827072
Sensitivity of F: 0.012887
Sensitivity of Q: 0.142857


In [45]:
#model: BMF
class BayesianMatrixFactorization():
    """
    Bayesian Matrix Factorization model
    R = PxQ
    p ~ N(p|0, alpha^(-1)I)
    q ~ N(q|0, alpha^(-1)I)
    r = p @ q
    t ~ N(r|p @ q, beta^(-1))
    """

    def __init__(self, alpha_p:float=1., alpha_q:float=1., beta:float=1.):
        """
        ----------
        n_u, n_i: the number of users and items, respectively.
        k : the number of latent factors
        """
        self.alpha_p = alpha_p
        self.alpha_q = alpha_q
        self.beta = beta
        #posterior of p,q 
        self.pos_mean_p = None
        self.pos_precision_p = None
        self.pos_mean_q = None
        self.pos_precision_q = None

    def fit(self, R:np.ndarray, k:int=5):
        """
        bayesian update of parameters given training dataset
        Parameters
        ----------
        R : (u,i) np.ndarray
            training data independent variable, u is the number of users, i is the number of items.
        k : int, the number of latent factors.
        """
        #1. generate matrices P, Q
        P = np.random.normal(0,self.alpha_p,(R.shape[0],k))#uxk
        Q = np.random.normal(0,self.alpha_q,(R.shape[1],k))#ixk
        #2.calculate the posterior with analytical solution
        self.pos_precision_p = self.alpha_p + self.beta * Q @ Q.T # ixi
        self.pos_mean_p = self.beta * R @ np.linalg.inv(self.pos_precision_p) @ Q # uxi,ixi,ixk -> uxk
        self.pos_precision_q = self.alpha_q + self.beta * P @ P.T # uxu
        self.pos_mean_q = self.beta * R.T @ np.linalg.inv(self.pos_precision_q) @ P # ixu,uxu,uxk -> ixk
        
    def predict(self, sample_size:int=None):
        """
        return mean  of predictive distribution
        Parameters
        ----------
        sample_size : int, optional
            number of samples to draw from the predictive distribution
            (the default is None, no sampling from the distribution)
        Returns
        -------
        R_pred : (u,i) np.ndarray
            mean of the predictive distribution
        R_pred_sample : (u,i,sample_size) np.ndarray
            samples from the predictive distribution
        """
        if sample_size is not None:
            R_sample = []
            for i in range(sample_size):
                p_sample, q_sample = [], []
                for k in range(self.pos_mean_p.shape[1]):#latent factors    
                    mean_p = self.pos_mean_p[:,k]
                    mean_q = self.pos_mean_q[:,k]
                    p_sample_k = np.random.multivariate_normal(mean_p, np.linalg.inv(self.pos_precision_q), size=1)
                    q_sample_k = np.random.multivariate_normal(mean_q, np.linalg.inv(self.pos_precision_p), size=1)
                    p_sample.append(p_sample_k.flatten())
                    q_sample.append(q_sample_k.flatten())
                R_sample.append(np.dot(np.array(p_sample).T, np.array(q_sample)))
            return  R_sample #uxi
        
        R_pred = self.pos_mean_p @ self.pos_mean_q.T #R = PxQ
        return R_pred #uxi

for topk in [5,10,15,20]:
    bmf = BayesianMatrixFactorization()
    bmf.fit(R=np.array(X_train), k=topk)
    R_pred = bmf.predict()
    #R_pred = np.array(bmf.predict(sample_size=10)).mean(axis=0)#sample weights.
    # buliding index of trainset
    #tstart = time.time()
    cpu_index = faiss.IndexFlatL2(trunc_len*2) #
    gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
    gpu_index.add(np.ascontiguousarray(R_pred, dtype=np.float32)) #add data(must be float32) to index
    #elapsed = time.time() - tstart    
    #print('Completed buliding index in %d seconds' % int(elapsed))
    #performance
    X_test = np.ascontiguousarray(X_test, dtype=np.float32)
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i in range(y_test.shape[0]):
        stype = np.array(y_test)[i]
        scores, neighbors = gpu_index.search(X_test[i:i+1], k=topk)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in neighbors.flatten():
            dtype = np.array(y_train)[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("MHR@{}={:.6f}, MAP@{}={:.6f}, MRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

MHR@5=0.195625, MAP@5=0.146205, MRR@5=0.760649
MHR@10=0.197250, MAP@10=0.180814, MRR@10=0.569168
MHR@15=0.179875, MAP@15=0.137433, MRR@15=0.531339
MHR@20=0.198063, MAP@20=0.186761, MRR@20=0.553211


In [59]:
def log_gaussian_loss(output, target, sigma, no_dim, sum_reduce=True):
    exponent = -0.5*(target - output)**2/sigma**2
    log_coeff = -no_dim*torch.log(sigma) - 0.5*no_dim*np.log(2*np.pi)
    
    if sum_reduce:
        return -(log_coeff + exponent).sum()
    else:
        return -(log_coeff + exponent)
    
class gaussian:
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma
        
    def loglik(self, weights):
        exponent = -0.5*(weights - self.mu)**2/self.sigma**2
        log_coeff = -0.5*(np.log(2*np.pi) + 2*np.log(self.sigma))
        
        return (exponent + log_coeff).sum()
    
class BayesLinear_Normalq(nn.Module):
    def __init__(self, input_dim, output_dim, prior):
        super(BayesLinear_Normalq, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.prior = prior
        
        self.weight_mus = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-0.01, 0.01))
        self.weight_rhos = nn.Parameter(torch.Tensor(self.input_dim, self.output_dim).uniform_(-3, -3))
        
    def forward(self, x):
        # sample gaussian noise for each weight
        weight_epsilons = Variable(self.weight_mus.data.new(self.weight_mus.size()).normal_())      
        # calculate the weight stds from the rho parameters
        weight_stds = torch.log(1 + torch.exp(self.weight_rhos))
        # calculate samples from the posterior from the sampled noise and mus/stds
        weight_sample = self.weight_mus + weight_epsilons*weight_stds
    
        output = torch.mm(x, weight_sample)
            
        # computing the KL loss term
        #reference: https://github.com/jojonki/AutoEncoders/blob/master/kl_divergence_between_two_gaussians.pdf
        prior_cov, varpost_cov = self.prior.sigma**2, weight_stds**2
        KL_loss = 0.5*(torch.log(prior_cov/varpost_cov)).sum() - 0.5*weight_stds.numel()
        KL_loss = KL_loss + 0.5*(varpost_cov/prior_cov).sum()
        KL_loss = KL_loss + 0.5*((self.weight_mus - self.prior.mu)**2/prior_cov).sum()
            
        return output, KL_loss


class BayesianNeuralNetwork(nn.Module):
    def __init__(self, input_dim=180, num_units=[128,32], output_dim=6):
        super(BayesianNeuralNetwork, self).__init__()
        
        # network with Bayesian linear.
        self.layer1 = BayesLinear_Normalq(input_dim, num_units[0], gaussian(0, 3))
        self.layer2 = BayesLinear_Normalq(num_units[0], num_units[1], gaussian(0, 3))
        self.layer3 = BayesLinear_Normalq(num_units[1], output_dim, gaussian(0, 3))
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
        # noise
        self.log_noise = nn.Parameter(torch.cuda.FloatTensor([3]))
    
    def forward(self, x):
        
        KL_loss_total = 0
        #x = x.view(-1, self.input_dim)
        x = x.view(x.size(0),-1) 
        #layer1
        x, KL_loss = self.layer1(x)
        x = self.activation(x)
        KL_loss_total = KL_loss_total + KL_loss
        #layer2
        x, KL_loss = self.layer2(x)
        x = self.activation(x) 
        KL_loss_total = KL_loss_total + KL_loss
        #layer3
        out, KL_loss = self.layer3(x)
        KL_loss_total = KL_loss_total + KL_loss
        
        return x, out, KL_loss_total


#define model
model = BayesianNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
batchSize = 100
num_batches = len(y_train) // batchSize
for epoch in range(10):#iteration
    #train model 
    losses = []
    for i in range(num_batches):
        min_idx = i * batchSize
        max_idx = np.min([len(y_train), (i+1)*batchSize])
        X_batch = torch.from_numpy(np.array(X_train[min_idx: max_idx]).astype(np.float32)).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(np.array(y_train[min_idx: max_idx]).astype(np.float32)).type(torch.LongTensor).cuda()
        optimizer.zero_grad()
        #forward
        _, out, KL_loss = model(X_batch)#adjust channel to the second
        out = F.log_softmax(out)
        fit_loss = F.nll_loss(out, Y_batch)
        loss = (KL_loss+fit_loss)/batchSize
        loss.backward()
        optimizer.step()
        sys.stdout.write('\r {} / {} : loss = {}'.format(i, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    
#release gpu memory
#model = model.cpu()
#torch.cuda.empty_cache()
#hash code of train data from model
num_batches = len(y_train) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(y_train), (i+1)*batchSize])
    X_batch = torch.from_numpy(np.array(X_train[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch, _, _ = model(X_batch)#forword
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(y_test) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(y_test), (i+1)*batchSize])
    X_batch = torch.from_numpy(np.array(X_test[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch, _, _ = model(X_batch)#forword
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#Evaluate model
cpu_index = faiss.IndexFlatL2(np.array(trF).shape[1]) #index with Faiss
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu  usable
gpu_index.add(np.array(trF).astype('float32')) #add data to index
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = np.array(y_test)[i]
        scores, neighbors = gpu_index.search(np.array(teF)[i:i+1].astype('float32'), k=topk)
        #map_item_score = {}
        #for j, trVal in enumerate(trF):
        #    map_item_score[j] = pdist(np.vstack([teVal,trVal]),'cosine')
        #ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors.flatten():
            dtype = np.array(y_train)[i]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 15 / 720 : loss = 986.149048

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:102: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


 719 / 720 : loss = 801.465942Eopch:     1 mean_loss = 895.234918
 719 / 720 : loss = 622.016052Eopch:     2 mean_loss = 710.581473
 719 / 720 : loss = 458.642914Eopch:     3 mean_loss = 538.559606
 719 / 720 : loss = 318.922821Eopch:     4 mean_loss = 386.437353
 719 / 720 : loss = 208.740997Eopch:     5 mean_loss = 261.029209
 719 / 720 : loss = 128.001282Eopch:     6 mean_loss = 165.658108
 719 / 720 : loss = 74.2212452Eopch:     7 mean_loss = 99.096252
 719 / 720 : loss = 25.279182Eopch:     9 mean_loss = 31.754674
 719 / 720 : loss = 13.078069Eopch:    10 mean_loss = 18.094523
 79 / 80 0 mHR@5=0.182500, mAP@5=0.182500, mRR@5=1.000000
mHR@10=0.182500, mAP@10=0.182500, mRR@10=1.000000
mHR@15=0.182500, mAP@15=0.182500, mRR@15=1.000000
mHR@20=0.182500, mAP@20=0.182500, mRR@20=1.000000


In [56]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self, input_dim=180, num_units=[128,32], output_dim=6):
        super(DeepNeuralNetwork, self).__init__()
        
        # network with  linear.
        self.layer1 = nn.Linear(input_dim,num_units[0])
        self.layer2 = nn.Linear(num_units[0],num_units[1])
        self.layer3 = nn.Linear(num_units[1],output_dim)
        
        # activation to be used between hidden layers
        self.activation = nn.ReLU(inplace = True)
    
    def forward(self, x):
        x = x.view(x.size(0),-1) 
        #layer1
        x = self.layer1(x)
        x = self.activation(x)
        #layer2
        x = self.layer2(x)
        x = self.activation(x) 
        #layer3
        out = self.layer3(x)
        
        return x, out


#define model
model = DeepNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
batchSize = 100
num_batches = len(y_train) // batchSize
for epoch in range(10):#iteration
    #train model 
    losses = []
    for i in range(num_batches):
        min_idx = i * batchSize
        max_idx = np.min([len(y_train), (i+1)*batchSize])
        X_batch = torch.from_numpy(np.array(X_train[min_idx: max_idx]).astype(np.float32)).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(np.array(y_train[min_idx: max_idx]).astype(np.float32)).type(torch.LongTensor).cuda()
        optimizer.zero_grad()
        #forward
        _, out = model(X_batch)#adjust channel to the second
        out = F.log_softmax(out)
        loss = F.nll_loss(out, Y_batch)
        loss.backward()
        optimizer.step()
        sys.stdout.write('\r {} / {} : loss = {}'.format(i, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    
#release gpu memory
#model = model.cpu()
#torch.cuda.empty_cache()
#hash code of train data from model
num_batches = len(y_train) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(y_train), (i+1)*batchSize])
    X_batch = torch.from_numpy(np.array(X_train[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch, _ = model(X_batch)#forword
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(y_test) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(y_test), (i+1)*batchSize])
    X_batch = torch.from_numpy(np.array(X_test[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch, _= model(X_batch)#forword
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#Evaluate model
cpu_index = faiss.IndexFlatL2(np.array(trF).shape[1]) #index with Faiss
gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu  usable
gpu_index.add(np.array(trF).astype('float32')) #add data to index
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = np.array(y_test)[i]
        scores, neighbors = gpu_index.search(np.array(teF)[i:i+1].astype('float32'), k=topk)
        #map_item_score = {}
        #for j, trVal in enumerate(trF):
        #    map_item_score[j] = pdist(np.vstack([teVal,trVal]),'cosine')
        #ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        #for j in ranklist:
        for j in neighbors.flatten():
            dtype = np.array(y_train)[i]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 39 / 720 : loss = 1.102698

/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


 719 / 720 : loss = 0.255489Eopch:     1 mean_loss = 0.567560
 719 / 720 : loss = 0.152731Eopch:     2 mean_loss = 0.270533
 719 / 720 : loss = 0.113899Eopch:     3 mean_loss = 0.217814
 719 / 720 : loss = 0.093981Eopch:     4 mean_loss = 0.187924
 719 / 720 : loss = 0.080152Eopch:     5 mean_loss = 0.166353
 719 / 720 : loss = 0.062912Eopch:     6 mean_loss = 0.149539
 719 / 720 : loss = 0.048382Eopch:     7 mean_loss = 0.136564
 719 / 720 : loss = 0.047562Eopch:     8 mean_loss = 0.126280
 719 / 720 : loss = 0.032049Eopch:     9 mean_loss = 0.118871
 719 / 720 : loss = 0.028015Eopch:    10 mean_loss = 0.113064
 79 / 80 0 mHR@5=0.182500, mAP@5=0.182500, mRR@5=1.000000
mHR@10=0.182500, mAP@10=0.182500, mRR@10=1.000000
mHR@15=0.182500, mAP@15=0.182500, mRR@15=1.000000
mHR@20=0.182500, mAP@20=0.182500, mRR@20=1.000000


In [58]:
x1 = torch.rand(10,180).cuda()
y = torch.LongTensor([0,1,2,3,4,5,0,1,3,4]).cuda()
model = DeepNeuralNetwork().cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
for epoch in range(10):
    optimizer.zero_grad()
    
    _,out = model(x1)
    out = F.log_softmax(out)
    loss = F.nll_loss(out, y)
    loss.backward()
    optimizer.step()
    sys.stdout.write('\r {} : loss = {}'.format(epoch, float('%0.6f'%loss.item())))
    #sys.stdout.flush()
#output
x2 = torch.rand(2,180).cuda()
x2,_ = model(x2)
#print (x2)
print (x2.size())

 9 : loss = 1.61822torch.Size([2, 32])


/root/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
